# Week 6 Assignment

## Environment Review

In [1]:
import sys

sys.version

'3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 23:51:54) \n[GCC 7.3.0]'

In [4]:
%%bash

ls
echo ""
cd necessary_scripts/
ls
echo ""
echo "----------------------"
echo "Line Count of getMutations"
wc -l getMutations.py
echo ""
echo "----------------------"
echo "Line Count of parseFastq.py"
wc -l parseFastq.py

dgorgon_reference.fa
harrington_clinical_data.txt
hawkins_pooled_sequences.fastq
necessary_scripts
Week6_Lay_that_Pipe.ipynb

faker.sorted.bam
faker.sorted.bam.bai
getMutations.py
parseFastq.py

----------------------
Line Count of getMutations
29 getMutations.py

----------------------
Line Count of parseFastq.py
92 parseFastq.py


### Initial Thoughts

These files are just too big, to look cat them out above, let's bring these files into jupyter

### hawking_pooled_sequences.fastq

In [7]:
%%bash

head -10 hawkins_pooled_sequences.fastq

@seq13534-419
GCAGTAGCGGTCATAAGTGGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGTGTCGTGAGGTGACGTCCGTCACTGGACGAA
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIDIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIFFFFDFFDFFDDFDFDFFFFDDFFDDFDDFF
@seq86249-867
GGATTAGCGGTCATAAGTCGTACATTACGAGATTCGGAGTACCATAGATTCGCATGAATCCCTGTGGATACGAGAGTGTGAGATATATGTACGCCAATCCAGTGTGATACCCATGAGATTTAGGACCGATGATGGTTGAGGACCAAGGATTGACCCGATGGATGCAGATTTGACCCCAGATAGAATAAATGCGATGAGATGATTTGGCCGATAGATAGATAGAGGTCAGTATAACCTCTCAAAGCTTTATCTACGGATGGATCCGCGC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIDIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

In [16]:
%%bash

cat harrington_clinical_data.txt

Name	Color	Barcode
Abbey	Black	GCAGA
Adelaide	Orange	CTTGA
Alana	Black	AGCTA
Arabella	Black	TTGAG
Arianna	Orange	TCCGC
Bobby	Black	CCCAC
Brayden	Black	GCAGT
Bridget	Orange	CGGTT
Bruce	Orange	ACGGA
Byron	Orange	AGCGA
Cali	Orange	TTTTT
Camilla	Orange	ACAGG
Case	Black	GGGAT
Cason	Black	AACTG
Damien	Black	TGCAA
Dania	Black	CATCC
Dereon	Orange	ACACT
Destiney	Orange	CCGTG
Dominik	Black	GGATT
Gabrielle	Orange	CGAAG
Giancarlo	Orange	TATGG
Greta	Black	CCAAC
Israel	Orange	CCTTT
Javion	Orange	TAGCT
Jaylen	Black	TCCGG
Jazlyn	Yellow	AGTGA
Jordyn	Black	CCACC
Jordyna	Yellow	CATAT
Joshua	Black	TACGT
Justus	Black	GTTCA
Kamden	Orange	CCGTA
Karlee	Yellow	GTCGT
Kayla	Yellow	CATAA
Keaton	Green	GCTAA
Kian	Green	GGAAG
Kirsten	Black	TGTGA
Libby	Yellow	AGTCA
Lorelei	Orange	GCGGA
Mariela	Yellow	CGGAG
Martin	Green	CAATT
Mckayla	Green	AGAGT
Mckenzie	Black	GAAAA
Natalia	Green	GATTC
Santiago	Orange	TGTAT
Sarai	Black	CTAGT
Serena	Green	GGTAA
Shelby	Yellow	AAATG
Sienna	Green	GACCT
Terry	Orange	AACCC
Tristian	Black	TG

### getMutations.py

pysam needs python 3.6 downgrading from 3.8 now

In [13]:
%%writefile getMutations.py
import pysam

#This is a slightly modified version from here: https://pysam.readthedocs.io/en/latest/api.html
# What is a pileup? It is "piling up" all the reads that have mapped to a given position of your reference.
# It is a useful way to see what reads have a mutation and what don't. 

def pileup():
    #test file, replaced with the sorted.bam you are using. Make sure it is indexed! 
    #(Use samtools index yourbam.sorted.bam)
    samfile = pysam.AlignmentFile("/home/rbif/week6/necessary_scripts/faker.sorted.bam", "rb")

    #Since our reference only has a single sequence, we're going to pile up ALL of the reads. 
    #Usually you would do it in a specific region (such as chromosome 1, position 1023 to 1050 for example)
    for pileupcolumn in samfile.pileup():
        print ("coverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))
        #use a dictionary to count up the bases at each position
        ntdict = {}
        for pileupread in pileupcolumn.pileups:
            if not pileupread.is_del and not pileupread.is_refskip:
                # You can uncomment the below line to see what is happening in the pileup. 
                # print ('\tbase in read %s = %s' % (pileupread.alignment.query_name, pileupread.alignment.query_sequence[pileupread.query_position]))
                base = pileupread.alignment.query_sequence[pileupread.query_position]
                
                ########## ADD ADDITIONAL CODE HERE ############# 
                # Populate the ntdict with the counts of each base 
                # This dictionary will hold all of the base read counts per nucletoide per position.
                # Use the dictionary to calculate the frequency of each site, and report it if if the frequency is NOT  100% / 0%. 
                #############################################
        print (ntdict)
    samfile.close()

if __name__=="__main__":
    pileup()

Writing getMutations.py


### parseFastq.py

In [24]:
%%writefile parseFastq.py

import argparse
#Example use is 
# python parseFastq.py --fastq /home/rbif/week6/hawkins_pooled_sequences.fastq


################################################
# You can use this code and put it in your own script
class ParseFastQ(object):
    """Returns a read-by-read fastQ parser analogous to file.readline()"""
    def __init__(self,filePath,headerSymbols=['@','+']):
        """Returns a read-by-read fastQ parser analogous to file.readline().
        Exmpl: parser.next()
        -OR-
        Its an iterator so you can do:
        for rec in parser:
            ... do something with rec ...
 
        rec is tuple: (seqHeader,seqStr,qualHeader,qualStr)
        """
        if filePath.endswith('.gz'):
            self._file = gzip.open(filePath)
        else:
            self._file = open(filePath, 'rU')
        self._currentLineNumber = 0
        self._hdSyms = headerSymbols
         
    def __iter__(self):
        return self
     
    def __next__(self):
        """Reads in next element, parses, and does minimal verification.
        Returns: tuple: (seqHeader,seqStr,qualHeader,qualStr)"""
        # ++++ Get Next Four Lines ++++
        elemList = []
        for i in range(4):
            line = self._file.readline()
            self._currentLineNumber += 1 ## increment file position
            if line:
                elemList.append(line.strip('\n'))
            else: 
                elemList.append(None)
         
        # ++++ Check Lines For Expected Form ++++
        trues = [bool(x) for x in elemList].count(True)
        nones = elemList.count(None)
        # -- Check for acceptable end of file --
        if nones == 4:
            raise StopIteration
        # -- Make sure we got 4 full lines of data --
        assert trues == 4,\
               "** ERROR: It looks like I encountered a premature EOF or empty line.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._currentLineNumber)
        # -- Make sure we are in the correct "register" --
        assert elemList[0].startswith(self._hdSyms[0]),\
               "** ERROR: The 1st line in fastq element does not start with '%s'.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._hdSyms[0],self._currentLineNumber) 
        assert elemList[2].startswith(self._hdSyms[1]),\
               "** ERROR: The 3rd line in fastq element does not start with '%s'.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._hdSyms[1],self._currentLineNumber) 
        # -- Make sure the seq line and qual line have equal lengths --
        assert len(elemList[1]) == len(elemList[3]), "** ERROR: The length of Sequence data and Quality data of the last record aren't equal.\n\
               Please check FastQ file near line number %s (plus or minus ~4 lines) and try again**" % (self._currentLineNumber) 
         
        # ++++ Return fatsQ data as tuple ++++
        return tuple(elemList)
##########################################################################



if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("-f", "--fastq", required=True, help="Place fastq inside here")
    args = parser.parse_args()

    #This is an example of how to use the function in your own code
    
    fastqfile = ParseFastQ(args.fastq)

    #A fastq read contains 4 lines
    for fastq_obj in fastqfile:
        #This fastq_obj is a tuple that has length of 4 and corresponds to those 4 lines
        #This is the header
        print(fastq_obj[0])
        #This is the sequence
        print(fastq_obj[1])
        #This is the separator
        print(fastq_obj[2])
        #This is the quality score
        print(fastq_obj[3])
        
        #Just an indicator showing the fastq "blocks"
        print('*'*10 + '==='*10 + '*' * 10)

Overwriting parseFastq.py


In [25]:
%%bash 

python parseFastq.py --fastq /home/rbif/week6/hawkins_pooled_sequences.fastq

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Workflow outline

Important note: We will not be able to use anything from BioPython for this assignment

### Demultiplex the pooled fastq by barcode

Create a single python script that:
1. Reads in the pooled fastq*
2. Uses the barcode from clinical data to find matches
3. Write to file
4. Trims barcode from start
5. Trims consequtive D or F from quality scores.
6. Set in foder with {name}_trimmed.fastq

In [20]:
#%%writefile pipeline.py


class Main:
    def __init__(self):
        print("hello!")
        
    def trimmer(self)
    
if __name__== "__main__":
    Main()

hello!
